In [21]:
#Import the relevand section of the NLP library
import nltk
nltk.download('punkt')
from nltk import word_tokenize
from nltk import sent_tokenize

import re

from mpl_toolkits.mplot3d import Axes3D
from sklearn.manifold import TSNE

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn import metrics

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

[nltk_data] Downloading package punkt to /Users/Alex/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [146]:
seinfeld_directory = 'Seinfeld_Episodes/'

seinfeld_season_1_episodes = ['Season_1/S01_E01_The_Seinfeld_Chronicles.txt', 'Season_1/S01_E02_The_Robbery.txt',
                             'Season_1/S01_E03_Male_Unbonding.txt', 'Season_1/S01_E04_The_Stock_Tip.txt']

seinfeld_season_5_episodes = ['Season_5/S05_E01_The_Mango.txt', 'Season_5/S05_E02_The_Puffy_Shirt.txt',
                              'Season_5/S05_E03_The_Glasses.txt', 'Season_5/S05_E04_The_Sniffing_Accountant.txt',
                              'Season_5/S05_E05_The_Bris.txt', 'Season_5/S05_E06_The_Lip_Reader.txt',
                              'Season_5/S05_E07_The_Non_Fat_Yogurt.txt', 'Season_5/S05_E08_The_Barber.txt',
                              'Season_5/S05_E09_The_Masseuse.txt', 'Season_5/S05_E10_The_Cigar_Store_Indian.txt',
                              'Season_5/S05_E11_The_Conversion.txt', 'Season_5/S05_E12_The_Stall.txt',
                              'Season_5/S05_E13_The_Dinner_Party.txt', 'Season_5/S05_E14_The_Marine_Biologist.txt',
                              'Season_5/S05_E15_The_Pie.txt', 'Season_5/S05_E16_The_Stand-In.txt',
                              'Season_5/S05_E17_The_Wife.txt', 'Season_5/S05_E18_The_Raincoats_Part_1.txt',
                              'Season_5/S05_E19_The_Raincoats_Part_2.txt', 'Season_5/S05_E20_The_Fire.txt',
                              'Season_5/S05_E21_The_Hamptons.txt', 'Season_5/S05_E22_The Opposite.txt']

curb_directory = 'Curb_Episodes/'

curb_season_3_episodes = ["Season_3/S03_E01_Chet's_Shirt.txt", 'Season_3/S03_E02_The_Benadryl_Brownie.txt',
                              'Season_3/S03_E03_Club_Soda_and_Salt .txt', 'Season_3/S03_E04_The_Nanny_from_Hell.txt',
                              'Season_3/S03_E05_The_Terrorist_Attack.txt', 'Season_3/S03_E06_The_Special_Section.txt',
                              'Season_3/S03_E07_The_Corpse-Sniffing_Dog.txt', 'Season_3/S03_E08_Krazee-Eyez_Killa.txt',
                              'Season_3/S03_E09_Mary,_Joseph_and_Larry.txt', 'Season_3/S03_E10_The_Grand_Opening.txt']

curb_season_4_episodes = ["Season_4/S04_E01_Mel's_Offer.txt", "Season_4/S04_E02_Ben's_Birthday_Party.txt",
                         'Season_4/S04_E03_The_Blind_Date.txt', 'Season_4/S04_E04_The_Weatherman.txt',
                         'Season_4/S04_E05_The_5_Wood.txt', 'Season_4/S04_E06_The_Car_Pool_Lane.txt',
                         'Season_4/S04_E07_The_Surrogate.txt', 'Season_4/S04_E08_Wandering_Bear.txt',
                         'Season_4/S04_E09_The_Survivor.txt', 'Season_4/S04_E10_Opening_Night.txt']

curb_season_7_episodes = ["Season_7/S07_E01_Funkhouser's_Crazy_Sister.txt", 'Season_7/S07_E02_Vehicular_Fellatio.txt',
                         'Season_7/S07_E03_The_Reunion.txt', 'Season_7/S07_E04_The_Hot_Towel.txt',
                         'Season_7/S07_E05_Denise_Handicapped.txt', 'Season_7/S07_E06_The_Bare_Midriff.txt',
                         'Season_7/S07_E07_The_Black_Swan.txt', 'Season_7/S07_E08_Officer_Krupke.txt',
                         'Season_7/S07_E09_The_Table_Read.txt', 'Season_7/S07_E10_Seinfeld.txt']



In [147]:
def convert_file_names_to_dictionary(directory_name, list_of_file_names):

    episode_info = []
    
    show_name = " ".join(directory_name.split('/')[0].split('_')[:-1])
    
    for i in list_of_file_names:
        
        season = int(" ".join(i.split('/')[0].split('_')[1:]))
        
        i_contents = i.split('/')[1].split('_')
        
        episode = int(i_contents[1][1:])
        
        episode_name = " ".join(i_contents[2:])[:-4]
        
        episode_info.append({'show_name': show_name, 'season': season, 'episode': episode, 'episode_name': episode_name, 'file_path': directory_name + i})
        
    return episode_info

In [148]:
convert_file_names_to_dictionary(seinfeld_directory, seinfeld_season_1_episodes)

[{'show_name': 'Seinfeld',
  'season': 1,
  'episode': 1,
  'episode_name': 'The Seinfeld Chronicles',
  'file_path': 'Seinfeld_Episodes/Season_1/S01_E01_The_Seinfeld_Chronicles.txt'},
 {'show_name': 'Seinfeld',
  'season': 1,
  'episode': 2,
  'episode_name': 'The Robbery',
  'file_path': 'Seinfeld_Episodes/Season_1/S01_E02_The_Robbery.txt'},
 {'show_name': 'Seinfeld',
  'season': 1,
  'episode': 3,
  'episode_name': 'Male Unbonding',
  'file_path': 'Seinfeld_Episodes/Season_1/S01_E03_Male_Unbonding.txt'},
 {'show_name': 'Seinfeld',
  'season': 1,
  'episode': 4,
  'episode_name': 'The Stock Tip',
  'file_path': 'Seinfeld_Episodes/Season_1/S01_E04_The_Stock_Tip.txt'}]

In [149]:
seinfeld_dict_season_1 = convert_file_names_to_dictionary(seinfeld_directory, seinfeld_season_1_episodes)

seinfeld_dict_season_5 = convert_file_names_to_dictionary(seinfeld_directory, seinfeld_season_5_episodes)

curb_dict_season_3 = convert_file_names_to_dictionary(curb_directory, curb_season_3_episodes)

curb_dict_season_4 = convert_file_names_to_dictionary(curb_directory, curb_season_4_episodes)

curb_dict_season_7 = convert_file_names_to_dictionary(curb_directory, curb_season_7_episodes)

shows_dict = seinfeld_dict_season_1 + seinfeld_dict_season_5 + curb_dict_season_3 + curb_dict_season_4 + curb_dict_season_7

In [150]:
df = pd.DataFrame(shows_dict)
df

,episode,episode_name,file_path,season,show_name
0,1,The Seinfeld Chronicles,Seinfeld_Episodes/Season_1/S01_E01_The_Seinfel...,1,Seinfeld
1,2,The Robbery,Seinfeld_Episodes/Season_1/S01_E02_The_Robbery...,1,Seinfeld
2,3,Male Unbonding,Seinfeld_Episodes/Season_1/S01_E03_Male_Unbond...,1,Seinfeld
3,4,The Stock Tip,Seinfeld_Episodes/Season_1/S01_E04_The_Stock_T...,1,Seinfeld
4,1,The Mango,Seinfeld_Episodes/Season_5/S05_E01_The_Mango.txt,5,Seinfeld
5,2,The Puffy Shirt,Seinfeld_Episodes/Season_5/S05_E02_The_Puffy_S...,5,Seinfeld
6,3,The Glasses,Seinfeld_Episodes/Season_5/S05_E03_The_Glasses...,5,Seinfeld
7,4,The Sniffing Accountant,Seinfeld_Episodes/Season_5/S05_E04_The_Sniffin...,5,Seinfeld
8,5,The Bris,Seinfeld_Episodes/Season_5/S05_E05_The_Bris.txt,5,Seinfeld
9,6,The Lip Reader,Seinfeld_Episodes/Season_5/S05_E06_The_Lip_Rea...,5,Seinfeld


In [162]:
df.loc[53].file_path

'Curb_Episodes/Season_7/S07_E08_Officer_Krupke.txt'

In [163]:
def open_file(file_path):

    with open(file_path, 'r') as file:
        raw_text = file.read().replace('\n', ' ')
    
    return raw_text

In [166]:
open_file(df.loc[54].file_path)

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x8e in position 12311: invalid start byte

In [9]:
raw_text_episode_2 = open_file(df.loc[1].file_path)

In [10]:
def cleaned_episode(raw_text):
    
    raw_text_no_notes = re.sub("[\(\[].*?[\)\]]", "", raw_text)

    for symbol in "*,.?!''\n":
        raw_text_no_notes = raw_text_no_notes.replace(symbol, '').lower()
  
    cleaned_text = raw_text_no_notes.split(" ")    
    
    for i in cleaned_text:
        
        if i.endswith(':') == True or i == '' or i == ' ':
            cleaned_text.remove(i)
            
        i = i.replace('.', '')
        i = i.replace('?', '')
        i = i.replace('!', '')

     
    return cleaned_text

In [11]:
for i in df.file_path:
    episode_text = open_file(i)
    print(cleaned_episode(episode_text)[:20])

['a', 'female', 'orgasm', 'is', 'kinda', 'like', 'the', 'bat', 'cave', 'a', 'very', 'few', 'people', 'know', 'where', 'it', 'is', 'and', 'if', 'youre']
['i', 'cant', 'believe', 'this', 'oh', 'it', 'wont', 'be', 'for', 'that', 'long', 'how', 'can', 'i', 'do', 'this', 'how', 'can', 'i', 'move']
['i', 'never', 'get', 'enough', 'sleep', 'i', 'stay', 'up', 'late', 'at', 'night', 'cause', 'im', 'night', 'guy', 'night', 'guy', 'wants', 'to', 'stay']
['jerry', 'george', 'and', 'elaine', 'at', 'monks', 'caf', 'so', 'does', 'he', 'like', 'you', 'what', 'do', 'you', 'think', 'you', 'like', 'him', 'yeah']
['act', 'one', 'scene', 'a', 'int', 'hospital', 'room', '-', 'day', 'jerry', 'elaine', 'and', 'george', 'are', 'visiting', 'with', 'new', 'parents', 'myra', 'and']
['at', 'the', 'comedy', 'club', 'professional', 'tennis', 'to', 'me', 'i', 'dont', 'understand', 'all', 'the', 'shushing', 'why', 'are', 'they', 'always', 'shushing', 'shh']
['jerry', 'ive', 'always', 'been', 'a', 'big', 'fan', 'of', '

# Part Two: tokenize/stemming/lemmatization

## Tokenize

In [12]:
episode_1_text = open_file(df.loc[0].file_path)
episode_1_text[:500]

"[location: nightclub] JERRY: A female orgasm is kinda like the bat cave. A very few people know where it is and if you're lucky enough to see it you probably don't know how you got there and you can't find you way back after you left. You know there are two types of female orgasm: the real and the fake. And I'll tell you right now, as a man, we don't know. We do not know, because to man sex is like a car accident and determining the female orgasm is like being asked 'What did you see after the c"

In [13]:
cleaned_episode_1_text = cleaned_episode(episode_1_text)
" ".join(cleaned_episode_1_text[:500])

'a female orgasm is kinda like the bat cave a very few people know where it is and if youre lucky enough to see it you probably dont know how you got there and you cant find you way back after you left you know there are two types of female the real and the fake and ill tell you right now as a man we dont know we do not know because to man sex is like a car accident and determining the female orgasm is like being asked what did you see after the car went out of control i heard a lot of screeching sounds i remember i was facing the wrong way at one point and in the end my body was thrown clear so whats her name karin is she nice great so you like her i think so you dont know i cant tell anymore well do you feel anything feel whats that all right let me ask you when she comes over youre cleaning up a lot yeah youre just straightening up or youre cleaning cleaning you do the tub yeah on your knees ajax scrubbing the whole deal yeah okay i think youre in love tub is love tub is love so the

In [14]:
def tokenize(clean_text):
    joined_episode = ' '.join(clean_text)
    tokenized_episode = word_tokenize(joined_episode)
    
    return tokenized_episode

In [15]:
tokenize_episode_1_text = tokenize(cleaned_episode_1_text)
" ".join(tokenize_episode_1_text[:500])

'a female orgasm is kinda like the bat cave a very few people know where it is and if youre lucky enough to see it you probably dont know how you got there and you cant find you way back after you left you know there are two types of female the real and the fake and ill tell you right now as a man we dont know we do not know because to man sex is like a car accident and determining the female orgasm is like being asked what did you see after the car went out of control i heard a lot of screeching sounds i remember i was facing the wrong way at one point and in the end my body was thrown clear so whats her name karin is she nice great so you like her i think so you dont know i cant tell anymore well do you feel anything feel whats that all right let me ask you when she comes over youre cleaning up a lot yeah youre just straightening up or youre cleaning cleaning you do the tub yeah on your knees ajax scrubbing the whole deal yeah okay i think youre in love tub is love tub is love so the

## Lemmetization

In [16]:
#https://www.machinelearningplus.com/nlp/lemmatization-examples-python/

import nltk
from nltk.stem import WordNetLemmatizer 

# Init the Wordnet Lemmatizer
lemmatizer = WordNetLemmatizer()

In [17]:
tokenize_episode_1_text

['a',
 'female',
 'orgasm',
 'is',
 'kinda',
 'like',
 'the',
 'bat',
 'cave',
 'a',
 'very',
 'few',
 'people',
 'know',
 'where',
 'it',
 'is',
 'and',
 'if',
 'youre',
 'lucky',
 'enough',
 'to',
 'see',
 'it',
 'you',
 'probably',
 'dont',
 'know',
 'how',
 'you',
 'got',
 'there',
 'and',
 'you',
 'cant',
 'find',
 'you',
 'way',
 'back',
 'after',
 'you',
 'left',
 'you',
 'know',
 'there',
 'are',
 'two',
 'types',
 'of',
 'female',
 'the',
 'real',
 'and',
 'the',
 'fake',
 'and',
 'ill',
 'tell',
 'you',
 'right',
 'now',
 'as',
 'a',
 'man',
 'we',
 'dont',
 'know',
 'we',
 'do',
 'not',
 'know',
 'because',
 'to',
 'man',
 'sex',
 'is',
 'like',
 'a',
 'car',
 'accident',
 'and',
 'determining',
 'the',
 'female',
 'orgasm',
 'is',
 'like',
 'being',
 'asked',
 'what',
 'did',
 'you',
 'see',
 'after',
 'the',
 'car',
 'went',
 'out',
 'of',
 'control',
 'i',
 'heard',
 'a',
 'lot',
 'of',
 'screeching',
 'sounds',
 'i',
 'remember',
 'i',
 'was',
 'facing',
 'the',
 'wrong'

In [18]:
def lemmatize_text(tokenized_text):
    return ' '.join([lemmatizer.lemmatize(w) for w in tokenized_text])

# One Function To Process Text

In [20]:
def process_text(file_name):
    raw_episode_text = open_file(file_name)
    clean_episode_text = cleaned_episode(raw_episode_text)
    tokenize_episode_text = tokenize(clean_episode_text)
    lemmatize_episode_text = lemmatize_text(tokenize_episode_text)
    
    return lemmatize_episode_text

In [33]:
df['tokenized_text'] = df.file_path.apply(lambda x: process_text(x))
df.head()

,episode,episode_name,file_path,season,show_name,tokenized_text
0,1,The Mango,Seinfeld_Episodes/Season_5/S05_E01_The_Mango.txt,5,Seinfeld,a female orgasm is kinda like the bat cave a v...
1,2,The Puffy Shirt,Seinfeld_Episodes/Season_5/S05_E02_The_Puffy_S...,5,Seinfeld,i cant believe this oh it wont be for that lon...
2,3,The Glasses,Seinfeld_Episodes/Season_5/S05_E03_The_Glasses...,5,Seinfeld,i never get enough sleep i stay up late at nig...
3,4,The Sniffing Accountant,Seinfeld_Episodes/Season_5/S05_E04_The_Sniffin...,5,Seinfeld,jerry george and elaine at monk caf so doe he ...
4,5,The Bris,Seinfeld_Episodes/Season_5/S05_E05_The_Bris.txt,5,Seinfeld,act one scene a int hospital room - day jerry ...


# Part Three: Vectorizing the Text

In [22]:
#You Must pass CountVectorizer a series, not a dataframe
count_vectorizer = CountVectorizer()
